<a href="https://colab.research.google.com/github/isabellebouchard/ift6135/blob/master/Problem_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem 2

**Instructions**: For this part of the assignment we will train a convolutional network on MNIST
for 10 epochs using your favorite deep learning frameworks such as Pytorch of Tensor
ow. Plot the
train and valid errors at the end of each epoch for the model.
1. Come up with a CNN architecture with more or less similar number of parameters as MLP
trained in Problem 1 and describe it.

MLP in Problem 1:

*   two hidden layers
*   input data size is 784 and output is parameterized by a softmax of 10 classes
*   train using probability loss (cross entropy) and minimize this criterion to optimize the model parameters using SGD
*   non-linearity chosen as neuron activation (activation function): ?
*   learning rate: ?
*   mini-batch size: ?


2. Compare the performances of CNN vs MLP. Comment.

You could take reference from the architecture mentioned here: 
https://github.com/MaximumEntropy/welcome_tutorials/tree/pytorch/pytorch 




# CNN Architecture



# CNN vs. MLP 

## Number of parameters

**CNN**: [x] number of parameters 

**MLP with 2 hidden layers**: 935,936 number of parameters

input to h1 = 784 x 512 = 401,408

h1 to h2 = 512 x 1024 = 524,288

h2 to output = 1024 x 10 = 10,240

CNN's need significantly less parameters than MLP since they exhibit sparse connectivity and parameter sharing. 

**Sparse connectivity**: direct connections very sparse, but in the deeper layers, units are indirectly connected to all/most of the input image. 

**Parameter sharing**: each member of the kernel is used at every position of the input, meaning the same parameters are used at all input locations. Therefore you learn only one set of parameters. 

Since we are asked to build a CNN with similar number of parameters as the MLP trained in Problem 1, the CNN should outperform the MLP since it needs a significantly smaller number of weights to be able to match an MLP's performance. 

## Performance

If m represents the size of the input image, and n the size of the output image, then MLP requires mxn parameters with O(mxn) runtime. In CNNs, you limit the number of connections of each output to k, this means you only need kxn parameters with O(kxn) runtime. 


# References

List of references we can check out to increase our performance:

*   MNIST Image Class Tensorflow CNN 99.51% Test Accur.: https://www.kaggle.com/raoulma/mnist-image-class-tensorflow-cnn-99-51-test-acc
*   How to Choose CNN Architecture for MNIST: https://www.kaggle.com/cdeotte/how-to-choose-cnn-architecture-mnist
*   99.75% Accur: https://www.kaggle.com/cdeotte/25-million-images-0-99757-mnist/notebook

